# Capstone project

Phase 1

1) Webscrape the provided URL  - IMDB dataset:

https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1

2) Store the entire data in two different CSV files as per the given fields:

The first CSV file data contains :

Sno, Movie Name, Director Name, Duration, year, ratings, Metascore

Bifurcate the Director field into subfields as per the number of directors of the movie belongs to such as Director1, director2

The second CSV file contains the following:

Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

Bifurcate the stars field into 4 subfields as per the number of stars worked in the movie such as star1, star2, star3, star4

Bifurcate the genre into 3 subfields as per the number of genres the movie belongs to such as :

Genre1, genre2, genre3

# Webscraping the URL

In [28]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import time
import requests
import re

# Set path for webdriver

In [29]:
driver=webdriver.Chrome('chromedriver.exe')

In [30]:
driver.get('https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1')

In [31]:
serial_no=[]
movie_name=[]
director_name_1=[]
director_name_2=[]
director_name_3=[]
duration=[]
year=[]
ratings=[]
metascore=[]

#rating
star_1=[]
star_2=[]
star_3=[]
star_4=[]
votes=[]
genre_1=[]
genre_2=[]
genre_3=[]
gross_collection=[]
certification=[]
url=[]

In [32]:
try:
    for page in range(0,10):
        
        #To extract Serial no. , Movie name and year
        s=driver.find_elements(By.XPATH,'//h3[@class="lister-item-header"]')
        for i in s:
            serial_no.append(i.text.split(".")[0])
            movie_name.append(i.text.split(".")[1].split("(")[0].strip(" "))
            year.append(i.text.split('(')[-1].strip(")"))
            
        # To extract Duration
        dur=driver.find_elements(By.XPATH,'//span[@class="runtime"]')
        for i in dur:
            duration.append(i.text.split(" ")[0])
        
        # To extract metascore
        meta=driver.find_elements(By.XPATH,'//div[@class="ratings-bar"]')
        for i in meta:
            try:
                metascore.append(i.text.split("\n")[1].split(" ")[0])
            except:
                metascore.append("Nan")

        #To extract ratings
        rat=driver.find_elements(By.TAG_NAME,'strong')
        for i in rat[2:]:
            ratings.append(i.text)
            
        # To extract director names with sub-fields.
        dire=driver.find_elements(By.XPATH,'//div[@class="lister-item-content"]')
        for i in dire:
            director_text = i.text
            directors = re.findall(r"Director[s]?:\s+(.*)", director_text)
            if directors:             
                director_name_1.append(directors[-1].split("|")[0].split(",")[0])
                try:
                    director_name_2.append(directors[-1].split("|")[0].split(",")[1])
                except:
                    director_name_2.append("nan")
                try:
                    director_name_3.append(directors[-1].split("|")[0].split(",")[2])
                except:
                    director_name_3.append("nan")
                     # TO extract votes and gross collection
        v=driver.find_elements(By.XPATH,'//p[@class="sort-num_votes-visible"]')
        for i in v:
            votes.append(i.text.split(":")[1].split(" ")[1])
        for i in v:
            try:
                gross_collection.append(i.text.split("|")[1].split("$")[1].strip("M"))
            except:
                gross_collection.append("Nan")

        # To extract certification
        cer=driver.find_elements(By.XPATH,'//p[@class="text-muted "]')
        for i in cer:
            try:
                (i.text.split("|")[2]) # To verify extraction condition
                certification.append(i.text.split("|")[0])
            except:
                certification.append("Nan")

        # To extract genres with sub-fields
        gen=driver.find_elements(By.XPATH,'//span[@class="genre"]')
        for i in gen:
            genre_1.append(i.text.split(",")[0])
            try:
                genre_2.append(i.text.split(",")[1])
            except:
                genre_2.append("Nan")
            try:
                genre_3.append(i.text.split(",")[2])
            except:
                genre_3.append("Nan")


        # To extract stars names with sub-fields
        star=driver.find_elements(By.XPATH,'//div[@class="lister-item-content"]')
        for i in dire:
            director_text = i.text
            directors = re.findall(r"Director[s]?:\s+(.*)", director_text)
            if directors:    
                star_1.append(directors[-1].split(":")[1].split(",")[0])
                try:
                    star_2.append(directors[-1].split(":")[1].split(",")[1])
                except:
                    star_2.append("Nan")
                try:
                    star_3.append(directors[-1].split(":")[1].split(",")[2])
                except:
                    star_3.append("Nan")
                try:
                    star_4.append(directors[-1].split(":")[1].split(",")[3])
                except:
                    star_4.append("Nan")

        # To extract URL of all movies to extract popularity data which is given inside of each movie page.
        url_t =driver.find_elements(By.XPATH, '//h3[@class="lister-item-header"]//a')
        for i in url_t:
            url.append(i.get_attribute("href"))


        search_button=driver.find_element(By.XPATH,'//a[@class="lister-page-next next-page"]')
        search_button.click()
        time.sleep(1)
except:
    print("Extraction Complete")

In [33]:
#IMPORTING PANDAS
import pandas as pd
df=pd.DataFrame({"S.No.":serial_no, "Movie_Name":movie_name,"Duration":duration,"Year":year,"Rating":ratings,"Metascore":metascore, "Director_1":director_name_1,"Director_2":director_name_2,"Director_3":director_name_3})
df

,S.No.,Movie_Name,Duration,Year,Rating,Metascore,Director_1,Director_2,Director_3
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.1,86,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson
1,2,The Dark Knight,152,2008,9.0,84,Christopher Nolan,nan,nan
2,3,The Lord of the Rings: The Return of the King,201,2003,9.0,94,Peter Jackson,nan,nan
3,4,Inception,148,2010,8.8,74,Christopher Nolan,nan,nan
4,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92,Peter Jackson,nan,nan
...,...,...,...,...,...,...,...,...,...
495,496,Thor,115,2011,7.0,57,Kenneth Branagh,nan,nan
496,497,Underworld,121,2003,7.0,42,Len Wiseman,nan,nan
497,498,Independence Day,145,1996,7.0,59,Roland Emmerich,nan,nan
498,499,Ant-Man and the Wasp,118,2018,7.0,70,Peyton Reed,nan,nan


In [34]:
df.to_csv('imbd1.csv')

In [35]:
df2=pd.DataFrame({"Movie_Name":movie_name, "Star_1":star_1,"Star_2":star_2,"Star_3":star_3,"Star_4":star_4,"Genre_1":genre_1,"Genre_2":genre_2,"Genre_3":genre_3, "Gross_Collection":gross_collection,"Certification":certification,"votes":votes})
df2

,Movie_Name,Star_1,Star_2,Star_3,Star_4,Genre_1,Genre_2,Genre_3,Gross_Collection,Certification,votes
0,Spider-Man: Across the Spider-Verse,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,Animation,Action,Adventure,Nan,PG,"54,872"
1,The Dark Knight,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Action,Crime,Drama,534.86,PG-13,"2,720,281"
2,The Lord of the Rings: The Return of the King,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,Action,Adventure,Drama,377.85,PG-13,"1,888,273"
3,Inception,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Action,Adventure,Sci-Fi,292.58,PG-13,"2,414,702"
4,The Lord of the Rings: The Fellowship of the Ring,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,Action,Adventure,Drama,315.54,PG-13,"1,916,881"
...,...,...,...,...,...,...,...,...,...,...,...
495,Thor,Chris Hemsworth,Anthony Hopkins,Natalie Portman,Tom Hiddleston,Action,Fantasy,Nan,181.03,PG-13,"869,417"
496,Underworld,Kate Beckinsale,Scott Speedman,Shane Brolly,Michael Sheen,Action,Fantasy,Thriller,51.97,R,"277,101"
497,Independence Day,Will Smith,Bill Pullman,Jeff Goldblum,Mary McDonnell,Action,Adventure,Sci-Fi,306.17,PG-13,"586,847"
498,Ant-Man and the Wasp,Paul Rudd,Evangeline Lilly,Michael Peña,Walton Goggins,Action,Adventure,Comedy,216.65,PG-13,"425,560"


In [36]:
df2.to_csv('imbd2.csv')

In [37]:
# Loading the datasets from csv files.
df3=pd.read_csv("imbd1.csv")

df4=pd.read_csv("imbd2.csv")

In [38]:
df3.sample(3)

,Unnamed: 0,S.No.,Movie_Name,Duration,Year,Rating,Metascore,Director_1,Director_2,Director_3
49,49,50,Yojimbo,110,1961,8.2,93,Akira Kurosawa,NaN,NaN
58,58,59,Pirates of the Caribbean: The Curse of the Bla...,143,2003,8.1,63,Gore Verbinski,NaN,NaN
399,399,400,Friday Night Lights,118,2004,7.2,70,Peter Berg,NaN,NaN


In [39]:
df4.sample(3)

,Unnamed: 0,Movie_Name,Star_1,Star_2,Star_3,Star_4,Genre_1,Genre_2,Genre_3,Gross_Collection,Certification,votes
336,336,Face/Off,John Travolta,Nicolas Cage,Joan Allen,Alessandro Nivola,Action,Crime,Sci-Fi,112.23,R,"387,756"
244,244,Mortal World,Ahmet Mümtaz Taylan,Alper Kul,Sarp Apak,Irem Sak,Action,Comedy,Crime,Nan,Nan,"28,178"
79,79,Kaththi,Joseph Vijay,Samantha Ruth Prabhu,Neil Nitin Mukesh,Tota Roy Chowdhury,Action,Drama,Nan,Nan,Not Rated,"28,343"


Phase 2

Make two tables and corresponding columns provided in the above CSV files in SQLite DBMS. Insert all data of each CSV file in each of the created tables. Now start querying the table(s) in the  SQL workbench / SQLite database :

Table 1: Sno, MovieName, Director Name, Duration, genre, ratings

1)  Display all the details of movies created by directors Christopher and Matt Reeves.

2) Display all the details of movies with a duration of 140 minutes to 190 minutes.

3) Display all details of movies with ratings above 7 in ascending order.

4) Display all movie names in descending order.

5) Display movie name starts with ‘P’ and their rating is greater than 7.



Table 2: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

1) Display all movie names with star Arnold Schwarzenegger in ascending order.

2    2) Display all details of the movie with the highest number of votes.

3) Display movie names with gross collections in descending order.

4) Display the gross collection of movies with the star Arnold.

5) Display all details of movies with comedy and action genres.



Make subquery :

1) Display all details from both tables where movie names are the same.

2)   Display all movie names, Director, ratings, and gross collection where the genre is action.

3)      Display all details from both tables with the highest gross collection.

4)      Display all details from both tables with the highest ratings

5)      Display all details from both tables with the lowest gross collection and lowest ratings



Now once completed with queries in the SQLite database, then make the exact query solutions by using PANDAS SQL in the data frame.  Load the CSV data in a data frame and start making solutions for all the above 15 queries using PANDAS SQL. You may use concat or merge joins per the requirements basis to make 5 join queries.

In [40]:
import sqlite3

In [41]:
#Create table
db=sqlite3.connect('database.db')

In [42]:
connect= sqlite3.connect('my_database.db')

In [43]:
df3.to_sql('imbd1', connect, if_exists='replace', index=False)

500

In [44]:
df4.to_sql('imbd2', connect, if_exists='replace', index=False)

500

# For Table1
Display all the details of movies created by directors Christopher and Matt Reeves

In [45]:
a=connect.execute("SELECT * FROM imbd1 WHERE Director_1 IN ('Christopher Nolan ', 'Matt Reeves ');")
for i in a:
    print(i)

(1, 2, 'The Dark Knight', 152, 2008, 9.0, '84', 'Christopher Nolan ', None, None)
(3, 4, 'Inception', 148, 2010, 8.8, '74', 'Christopher Nolan ', None, None)
(21, 22, 'The Dark Knight Rises', 164, 2012, 8.4, '78', 'Christopher Nolan ', None, None)
(41, 42, 'Batman Begins', 140, 2005, 8.2, '70', 'Christopher Nolan ', None, None)
(130, 131, 'The Batman', 176, 2022, 7.8, '72', 'Matt Reeves ', None, None)
(136, 137, 'Dunkirk', 106, 2017, 7.8, '94', 'Christopher Nolan ', None, None)
(225, 226, 'Dawn of the Planet of the Apes', 130, 2014, 7.6, '79', 'Matt Reeves ', None, None)
(304, 305, 'War for the Planet of the Apes', 140, 2017, 7.4, '82', 'Matt Reeves ', None, None)
(322, 323, 'Tenet', 150, 2020, 7.3, '69', 'Christopher Nolan ', None, None)


2.Details of movies with a duration of 140 minutes to 190 minutes.

In [46]:
b=connect.execute("SELECT * FROM imbd1 WHERE Duration >= 140 AND Duration <= 190")
for i in b:
    print(i)

(0, 1, 'Spider-Man: Across the Spider-Verse', 140, 2023, 9.1, '86', 'Joaquim Dos Santos', ' Kemp Powers', ' Justin K. Thompson ')
(1, 2, 'The Dark Knight', 152, 2008, 9.0, '84', 'Christopher Nolan ', None, None)
(3, 4, 'Inception', 148, 2010, 8.8, '74', 'Christopher Nolan ', None, None)
(4, 5, 'The Lord of the Rings: The Fellowship of the Ring', 178, 2001, 8.8, '92', 'Peter Jackson ', None, None)
(5, 6, 'The Lord of the Rings: The Two Towers', 179, 2002, 8.8, '87', 'Peter Jackson ', None, None)
(8, 9, 'Soorarai Pottru', 153, 2020, 8.7, 'Nan', 'Sudha Kongara ', None, None)
(13, 14, 'Sita Ramam', 163, 2022, 8.6, 'Nan', 'Hanu Raghavapudi ', None, None)
(15, 16, 'Gladiator', 155, 2000, 8.5, '67', 'Ridley Scott ', None, None)
(18, 19, 'Avengers: Endgame', 181, 2019, 8.4, '78', 'Anthony Russo', ' Joe Russo ', None)
(19, 20, 'Avengers: Infinity War', 149, 2018, 8.4, '68', 'Anthony Russo', ' Joe Russo ', None)
(21, 22, 'The Dark Knight Rises', 164, 2012, 8.4, '78', 'Christopher Nolan ', None, 

Details of movies with ratings above 7 in ascending order

In [47]:
c=connect.execute("SELECT * FROM imbd1 WHERE Rating > 7 ORDER BY Rating ASC")
for i in c:
    print(i)

(430, 431, 'Furious 7', 137, 2015, 7.1, '67', 'James Wan ', None, None)
(431, 432, 'The Mummy', 124, 1999, 7.1, '48', 'Stephen Sommers ', None, None)
(432, 433, 'Wrath of Man', 119, 2021, 7.1, '57', 'Guy Ritchie ', None, None)
(433, 434, 'Mission: Impossible', 110, 1996, 7.1, '59', 'Brian De Palma ', None, None)
(434, 435, 'Free Guy', 115, 2021, 7.1, '62', 'Shawn Levy ', None, None)
(435, 436, 'Dredd', 95, 2012, 7.1, '60', 'Pete Travis ', None, None)
(436, 437, 'Everest', 121, 2015, 7.1, '64', 'Baltasar Kormákur ', None, None)
(437, 438, 'Tropic Thunder', 107, 2008, 7.1, '71', 'Ben Stiller ', None, None)
(438, 439, 'Prey', 100, 2022, 7.1, '71', 'Dan Trachtenberg ', None, None)
(439, 440, 'Man of Steel', 143, 2013, 7.1, '55', 'Zack Snyder ', None, None)
(440, 441, 'Iron Man 3', 130, 2013, 7.1, '62', 'Shane Black ', None, None)
(441, 442, 'Shooter', 124, 2007, 7.1, '53', 'Antoine Fuqua ', None, None)
(442, 443, "Pirates of the Caribbean: At World's End", 169, 2007, 7.1, '50', 'Gore Verbi

Details movie names in descending order

In [48]:
d=connect.execute("SELECT Movie_Name FROM imbd1 ORDER BY Movie_Name DESC")
for i in d:
    print(i)

('Zombieland',)
("Zack Snyder's Justice League",)
('Yojimbo',)
('X2: X-Men United',)
('X-Men: First Class',)
('X-Men: Days of Future Past',)
('X-Men',)
('Wrath of Man',)
('Wonder Woman',)
('Wolfwalkers',)
('Willow',)
('White Heat',)
('Where Eagles Dare',)
('We Were Soldiers',)
('Watchmen',)
('Warrior',)
('WarGames',)
('War for the Planet of the Apes',)
('War Horse',)
('Vishwaroopam',)
('Vikram Vedha',)
('Vikram',)
('Vanishing Point',)
('Vampire Hunter D: Bloodlust',)
('V for Vendetta',)
('Uri: The Surgical Strike',)
('Upgrade',)
('United 93',)
('Underworld',)
('Unbroken',)
('Udta Punjab',)
('True Lies',)
('Tropic Thunder',)
('Transformers',)
('Training Day',)
('Train to Busan',)
('Total Recall',)
('Tora! Tora! Tora!',)
('Top Gun: Maverick',)
('To Strike',)
('To Live and Die in L',)
('Thuppakki',)
('Three Kings',)
('Thor: Ragnarok',)
('Thor',)
('Thirteen Lives',)
('Thief',)
('They Live',)
('The Witcher: Nightmare of the Wolf',)
('The Wild Bunch',)
('The Way of the Dragon',)
('The Warrio

All movie name starts with ‘P’ and their rating is greater than 7

In [49]:
e=connect.execute("SELECT Movie_Name FROM imbd1 WHERE Movie_Name LIKE 'P%' AND Rating > 7")
for i in e:
    print(i)

('Princess Mononoke',)
('Paan Singh Tomar',)
('Pirates of the Caribbean: The Curse of the Black Pearl',)
('Predator',)
('Ponniyin Selvan: Part I',)
('Pushpa: The Rise - Part 1',)
('Police Story',)
('Predestination',)
("Pirates of the Caribbean: Dead Man's Chest",)
('Patriots Day',)
('Point Break',)
('Prey',)
("Pirates of the Caribbean: At World's End",)
('Planet Terror',)
('Payback',)


# Table 2

All movie names with star Arnold Schwarzenegger in ascending order

In [50]:
F=connect.execute("SELECT Movie_Name FROM imbd2 WHERE Star_1 = ' Arnold Schwarzenegger' or Star_2= ' Arnold Schwarzenegger' or Star_3 = ' Arnold Schwarzenegger' or Star_4= ' Arnold Schwarzenegger' ORDER BY Movie_Name ASC;")
for i in F:
    print(i)

('Predator',)
('Terminator 2: Judgment Day',)
('The Terminator',)
('Total Recall',)
('True Lies',)


All details of the movie with the highest number of votes.

In [51]:
G=connect.execute("SELECT * FROM imbd2 WHERE votes = (SELECT MAX(votes) FROM imbd2)")
for i in G:
    print(i)

(47, 'Gangs of Wasseypur', ' Manoj Bajpayee', ' Nawazuddin Siddiqui', ' Tigmanshu Dhulia', ' Richa Chadha', 'Action', ' Comedy', ' Crime', 'Nan', 'Not Rated ', '99,402')


Movie names with gross collections in descending order.

In [52]:
H=connect.execute("SELECT Movie_Name, Gross_Collection FROM imbd2 WHERE Gross_Collection IS NOT NULL ORDER BY Gross_Collection DESC")

for i in H:
    print(i)

('Spider-Man: Across the Spider-Verse', 'Nan')
('Soorarai Pottru', 'Nan')
('Harakiri', 'Nan')
('Sita Ramam', 'Nan')
('Kaithi', 'Nan')
('Asuran', 'Nan')
('Kantara', 'Nan')
('Raatchasan', 'Nan')
('Vikram', 'Nan')
('Shershaah', 'Nan')
('Guardians of the Galaxy Vol', 'Nan')
('Gangs of Wasseypur', 'Nan')
('Vikram Vedha', 'Nan')
('Yojimbo', 'Nan')
('K', 'Nan')
('The Mountain II', 'Nan')
('Neon Genesis Evangelion: The End of Evangelion', 'Nan')
('Sholay', 'Nan')
('The Hidden Fortress', 'Nan')
('White Heat', 'Nan')
('A Wednesday', 'Nan')
('Sarfarosh', 'Nan')
('Major', 'Nan')
('Hera Pheri', 'Nan')
('Thuppakki', 'Nan')
('Kaththi', 'Nan')
('John Wick: Chapter 4', 'Nan')
('Ip Man', 'Nan')
('Wolfwalkers', 'Nan')
('Sanjuro', 'Nan')
('Karthikeya 2', 'Nan')
('Andaz Apna Apna', 'Nan')
('Arjun Reddy', 'Nan')
('The Breath', 'Nan')
('1: Nenokkadine', 'Nan')
("Zack Snyder's Justice League", 'Nan')
('Baby', 'Nan')
('Airlift', 'Nan')
('Vishwaroopam', 'Nan')
('All Quiet on the Western Front', 'Nan')
('Thirtee

Display all details of movies with comedy and action genres

In [53]:
J= connect.execute("SELECT * FROM imbd2 WHERE Genre_1 IN ('Comedy', 'Action') or Genre_2 IN ('Comedy', 'Action') or Genre_3 IN ('Comedy', 'Action');")

for i in J:
    print(i)

(1, 'The Dark Knight', ' Christian Bale', ' Heath Ledger', ' Aaron Eckhart', ' Michael Caine', 'Action', ' Crime', ' Drama', '534.86', 'PG-13 ', '2,720,281')
(2, 'The Lord of the Rings: The Return of the King', ' Elijah Wood', ' Viggo Mortensen', ' Ian McKellen', ' Orlando Bloom', 'Action', ' Adventure', ' Drama', '377.85', 'PG-13 ', '1,888,273')
(3, 'Inception', ' Leonardo DiCaprio', ' Joseph Gordon-Levitt', ' Elliot Page', ' Ken Watanabe', 'Action', ' Adventure', ' Sci-Fi', '292.58', 'PG-13 ', '2,414,702')
(4, 'The Lord of the Rings: The Fellowship of the Ring', ' Elijah Wood', ' Ian McKellen', ' Orlando Bloom', ' Sean Bean', 'Action', ' Adventure', ' Drama', '315.54', 'PG-13 ', '1,916,881')
(5, 'The Lord of the Rings: The Two Towers', ' Elijah Wood', ' Ian McKellen', ' Viggo Mortensen', ' Orlando Bloom', 'Action', ' Adventure', ' Drama', '342.55', 'PG-13 ', '1,704,299')
(6, 'The Matrix', ' Keanu Reeves', ' Laurence Fishburne', ' Carrie-Anne Moss', ' Hugo Weaving', 'Action', ' Sci-Fi

# Display all details from both tables where movie names are the same.

In [54]:
K = connect.execute("SELECT * FROM imbd1 JOIN imbd2 on imbd1.Movie_Name = imbd2.Movie_Name")

for i in K:
    print(i)

(0, 1, 'Spider-Man: Across the Spider-Verse', 140, 2023, 9.1, '86', 'Joaquim Dos Santos', ' Kemp Powers', ' Justin K. Thompson ', 0, 'Spider-Man: Across the Spider-Verse', ' Shameik Moore', ' Hailee Steinfeld', ' Brian Tyree Henry', ' Luna Lauren Velez', 'Animation', ' Action', ' Adventure', 'Nan', 'PG ', '54,872')
(1, 2, 'The Dark Knight', 152, 2008, 9.0, '84', 'Christopher Nolan ', None, None, 1, 'The Dark Knight', ' Christian Bale', ' Heath Ledger', ' Aaron Eckhart', ' Michael Caine', 'Action', ' Crime', ' Drama', '534.86', 'PG-13 ', '2,720,281')
(2, 3, 'The Lord of the Rings: The Return of the King', 201, 2003, 9.0, '94', 'Peter Jackson ', None, None, 2, 'The Lord of the Rings: The Return of the King', ' Elijah Wood', ' Viggo Mortensen', ' Ian McKellen', ' Orlando Bloom', 'Action', ' Adventure', ' Drama', '377.85', 'PG-13 ', '1,888,273')
(3, 4, 'Inception', 148, 2010, 8.8, '74', 'Christopher Nolan ', None, None, 3, 'Inception', ' Leonardo DiCaprio', ' Joseph Gordon-Levitt', ' Ellio

All movie names, Director, ratings, and gross collection where the genre is action

In [55]:
M= connect.execute("SELECT imbd1.Movie_Name, imbd1.Director_1, imbd1.Rating, imbd2.Gross_Collection, imbd2.Genre_1 FROM imbd1, imbd2 WHERE (imbd2.Genre_1 = 'Action' ) AND imbd1.Movie_Name = imbd2.Movie_Name")

for i in M:
    print(i)

('The Dark Knight', 'Christopher Nolan ', 9.0, '534.86', 'Action')
('The Lord of the Rings: The Return of the King', 'Peter Jackson ', 9.0, '377.85', 'Action')
('Inception', 'Christopher Nolan ', 8.8, '292.58', 'Action')
('The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson ', 8.8, '315.54', 'Action')
('The Lord of the Rings: The Two Towers', 'Peter Jackson ', 8.8, '342.55', 'Action')
('The Matrix', 'Lana Wachowski', 8.7, '171.48', 'Action')
('Star Wars: Episode V - The Empire Strikes Back', 'Irvin Kershner ', 8.7, '290.48', 'Action')
('Soorarai Pottru', 'Sudha Kongara ', 8.7, 'Nan', 'Action')
('Star Wars: Episode IV - A New Hope', 'George Lucas ', 8.6, '322.74', 'Action')
('Terminator 2: Judgment Day', 'James Cameron ', 8.6, '204.84', 'Action')
('Seven Samurai', 'Akira Kurosawa ', 8.6, '0.27', 'Action')
('Harakiri', 'Masaki Kobayashi ', 8.6, 'Nan', 'Action')
('Sita Ramam', 'Hanu Raghavapudi ', 8.6, 'Nan', 'Action')
('Léon: The Professional', 'Luc Besson ', 8.5, '19.50',

Display all details from both tables with the highest gross collection

In [56]:
N= connect.execute("SELECT * FROM imbd1, imbd2 WHERE imbd2.Gross_Collection = (SELECT max(Gross_Collection) FROM imbd2) AND imbd1.Movie_Name = imbd2.Movie_Name")

for i in N:
    print(i)

(0, 1, 'Spider-Man: Across the Spider-Verse', 140, 2023, 9.1, '86', 'Joaquim Dos Santos', ' Kemp Powers', ' Justin K. Thompson ', 0, 'Spider-Man: Across the Spider-Verse', ' Shameik Moore', ' Hailee Steinfeld', ' Brian Tyree Henry', ' Luna Lauren Velez', 'Animation', ' Action', ' Adventure', 'Nan', 'PG ', '54,872')
(8, 9, 'Soorarai Pottru', 153, 2020, 8.7, 'Nan', 'Sudha Kongara ', None, None, 8, 'Soorarai Pottru', ' Suriya', ' Paresh Rawal', ' Urvashi', ' Aparna Balamurali', 'Action', ' Drama', 'Nan', 'Nan', 'TV-MA ', '119,545')
(12, 13, 'Harakiri', 133, 1962, 8.6, '85', 'Masaki Kobayashi ', None, None, 12, 'Harakiri', ' Tatsuya Nakadai', ' Akira Ishihama', ' Shima Iwashita', ' Tetsurô Tanba', 'Action', ' Drama', ' Mystery', 'Nan', 'Not Rated ', '61,060')
(13, 14, 'Sita Ramam', 163, 2022, 8.6, 'Nan', 'Hanu Raghavapudi ', None, None, 13, 'Sita Ramam', ' Dulquer Salmaan', ' Mrunal Thakur', ' Rashmika Mandanna', ' Sumanth', 'Action', ' Drama', ' Mystery', 'Nan', 'U ', '60,066')
(23, 24, '

Display all details from both tables with the highest ratings

In [57]:
P= connect.execute("SELECT * FROM imbd1, imbd2 WHERE imbd1.Rating = (SELECT MAX(Rating) FROM imbd1) AND imbd1.Movie_Name = imbd2.Movie_Name")

for i in P:
    print(i)

(0, 1, 'Spider-Man: Across the Spider-Verse', 140, 2023, 9.1, '86', 'Joaquim Dos Santos', ' Kemp Powers', ' Justin K. Thompson ', 0, 'Spider-Man: Across the Spider-Verse', ' Shameik Moore', ' Hailee Steinfeld', ' Brian Tyree Henry', ' Luna Lauren Velez', 'Animation', ' Action', ' Adventure', 'Nan', 'PG ', '54,872')


Display all details from both tables with the lowest gross collection and lowest ratings

In [58]:
P = connect.execute("SELECT * FROM imbd1, imbd2 WHERE (imbd2.Gross_Collection =(SELECT min(Gross_Collection) FROM imbd2) or imbd1.Rating=(SELECT min(Rating) FROM imbd1)) AND imbd1.Movie_Name = imbd2.Movie_Name")

for i in P:
    print(i)

(161, 162, "Knockin' on Heaven's Door", 87, 1997, 7.8, 'Nan', 'Thomas Jahn ', None, None, 161, "Knockin' on Heaven's Door", ' Til Schweiger', ' Jan Josef Liefers', ' Thierry van Werveke', ' Moritz Bleibtreu', 'Action', ' Crime', ' Comedy', '0.00', 'Not Rated ', '32,107')
(488, 489, 'Sisu', 91, 2022, 7.0, '70', 'Jalmari Helander ', None, None, 488, 'Sisu', ' Jorma Tommila', ' Aksel Hennie', ' Jack Doolan', ' Mimosa Willamo', 'Action', ' War', 'Nan', 'Nan', 'R ', '30,539')
(489, 490, 'Shazam!', 132, 2019, 7.0, '71', 'David F. Sandberg ', None, None, 489, 'Shazam!', ' Zachary Levi', ' Mark Strong', ' Asher Angel', ' Jack Dylan Grazer', 'Action', ' Adventure', ' Comedy', '140.37', 'PG-13 ', '363,932')
(490, 491, 'Transformers', 144, 2007, 7.0, '61', 'Michael Bay ', None, None, 490, 'Transformers', ' Shia LaBeouf', ' Megan Fox', ' Josh Duhamel', ' Tyrese Gibson', 'Action', ' Adventure', ' Sci-Fi', '319.25', 'PG-13 ', '652,136')
(491, 492, 'The Unbearable Weight of Massive Talent', 107, 2022